In [1]:
import pandas as pd
import sqlite3

In [2]:
con = sqlite3.connect("netflix.sqlite")
cur = con.cursor()
df = pd.read_sql("select * from netflix_titles", sqlite3.connect("netflix.sqlite"))

In [3]:
print(df['show_id'].shape[0])
print(len(df['show_id'].unique()))
df.sample(3)

6234
6234


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
6021,81008236,TV Show,COMEDIANS of the world,,"Neal Brennan, Chris D'Elia, Nicole Byer, Nick ...",United States,"January 1, 2019",2019,TV-MA,13 Seasons,"Stand-Up Comedy & Talk Shows, TV Comedies",This global stand-up comedy series features a ...
4117,80223271,Movie,Nothing to Lose 2,Alexandre Avancini,"Petrônio Gontijo, Day Mesquita, Beth Goulart, ...",Brazil,"December 7, 2019",2019,PG-13,97 min,"Dramas, Faith & Spirituality, International Mo...",As controversy surrounds the evangelical churc...
4460,80039602,Movie,Skin Trade,Ekachai Uekrongtham,"Dolph Lundgren, Tony Jaa, Ron Perlman, Celina ...","Thailand, Canada, United States","December 17, 2018",2014,R,96 min,Action & Adventure,A New Jersey detective whose family was killed...


In [4]:
actors_df = df['cast'].str.split(', ')

In [5]:
actors = set()
for line in actors_df:
    for actor in line:
        t = actor.strip()
        if t != '':
            actors.add(t)
len(actors)

27405

In [6]:
shows = df['title'].unique()

In [7]:
with con: 
    cur.execute("""
        CREATE TABLE IF NOT EXISTS show (
            id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
            name TEXT
        );
    """)

In [8]:
with con:
    for show in shows:
        cur.execute("INSERT INTO show (name) values(?)", (show,))

In [9]:
with con: 
    cur.execute("""
        CREATE TABLE IF NOT EXISTS actor (
            id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
            name TEXT
        );
    """)

In [10]:
with con:
    for actor in actors:
        cur.execute("INSERT INTO actor (name) values(?)", (actor,))

In [11]:
df_actor = pd.read_sql("select * from actor", sqlite3.connect("netflix.sqlite"), index_col='id')
df_actor

,name
id,
1,Ryuzo Hasuike
2,Scott Shepherd
3,Chris Evans
4,Melany Bennet
5,Jay Duffy
...,...
27401,Tomás Ottaviano
27402,Helge Schneider
27403,Petey Majik Nguyen


In [12]:
df_show = pd.read_sql("select * from show", sqlite3.connect("netflix.sqlite"), index_col='id')
df_show

,name
id,
1,Norm of the North: King Sized Adventure
2,Jandino: Whatever it Takes
3,Transformers Prime
4,Transformers: Robots in Disguise
5,#realityhigh
...,...
6168,Kikoriki
6169,Red vs. Blue
6170,Maron


In [13]:
with con: 
    cur.execute("""
        CREATE TABLE IF NOT EXISTS actor_show (
            actor TEXT,
            show_id INT
        );
    """)

In [14]:
%%time
count_last_upd = len(df)
for i in range(len(df) - count_last_upd, len(df)):
    for actor in actors_df.iloc[i]:
        if not actor:
            continue
        with con:
            cur.execute("INSERT INTO actor_show (actor, show_id) values(?, ?)", (actor, int(df.iloc[i]['show_id'])))

CPU times: total: 1min 22s
Wall time: 5min 43s


In [15]:
df_as = pd.read_sql("select * from actor_show", sqlite3.connect("netflix.sqlite"))
df_as

,actor,show_id
0,Alan Marriott,81145628
1,Andrew Toth,81145628
2,Brian Dobson,81145628
3,Cole Howard,81145628
4,Jennifer Cameron,81145628
...,...,...
44306,Courteney Cox,70153404
44307,Lisa Kudrow,70153404
44308,Matt LeBlanc,70153404
44309,Matthew Perry,70153404


In [16]:
with con: 
    cur.execute("""
        SELECT t1.actor as value_1,
               t2.actor as value_2,
               count(t1.show_id)
        FROM actor_show t1
        CROSS JOIN
               actor_show t2
        where t1.show_id = t2.show_id and t1.actor != t2.actor
        group by t1.actor, t2.actor
        order by count(t1.show_id) DESC
        limit 1
    """)
cur.fetchone()[:-1]

('John Paul Tremblay', 'Robb Wells')

In [17]:
cur.close()